In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
import xgboost as xgb
import sklearn.datasets
import gc

### Data Prep.

In [29]:
# train_X = pd.read_csv('music_data/train_X.csv')
train_X = pd.read_csv('music_data/train_X_date_converted.csv')
train_y = pd.read_csv('music_data/train_Y.csv')

# val_X = pd.read_csv('music_data/valid_X.csv')
val_X = pd.read_csv('music_data/valid_X_date_converted.csv')
val_y = pd.read_csv('music_data/valid_Y.csv')

# test_X = pd.read_csv('music_data/test_X.csv')
test_X = pd.read_csv('music_data/test_X_date_converted.csv')
test_y = pd.read_csv('music_data/test_Y.csv')

# train_tiny_X = pd.read_csv('music_data/train_tiny_X.csv')
train_tiny_X = pd.read_csv('music_data/train_tiny_X_date_converted.csv')
train_tiny_y = pd.read_csv('music_data/train_tiny_Y.csv')

# val_tiny_X = val_X[:1000].copy()
val_tiny_X = pd.read_csv('music_data/val_tiny_X_date_converted.csv')
val_tiny_y = val_y[:1000].copy()

In [11]:
# Observing reasonable date for date time conversion
# 2004 seems to be the earliest date. '1970-01-01' is essentially null value (1 in train).
print('train x', train_X['registration_init_time'].min(), train_X['expiration_date'].min())
print('val x', val_X['registration_init_time'].min(), val_X['expiration_date'].min())
print('test x', test_X['registration_init_time'].min(), test_X['expiration_date'].min())


# Not many records smaller than '2000-01-01'. Reasonable to not treat them specially
print(train_X['expiration_date'].apply(lambda date: date == '1970-01-01').sum(), '1970-01-01 in train set')
print(val_X['expiration_date'].apply(lambda date: date == '1970-01-01').sum(), '1970-01-01 in val set')

print(train_X['expiration_date'].apply(lambda date: date < '2000-01-01').sum(), 'smaller than 2000-01-01 in train set')
print(val_X['expiration_date'].apply(lambda date: date < '2000-01-01').sum(), 'smaller than 2000-01-01 in val set')

train x 2004-03-26 1970-01-01
val x 2004-03-26 1970-01-01
test x 2004-03-26 2004-10-16
1 1970-01-01 in train set
2 1970-01-01 in val set
1 smaller than 2000-01-01 in train set
2 smaller than 2000-01-01 in val set


In [ ]:
def date_to_int(column, base_date=datetime.strptime('2000-01-01', "%Y-%m-%d")):
    """
    Convert date to day counts since base_date for given columns.
    
    :param column: pandas column containing date representations as str value
    :param base_date: base date from which date is counted. i.e. 2000-01-02 will be day '1' comparing to '2000-01-01'
    """
    
    def date_diff(date):
        date = datetime.strptime(date, "%Y-%m-%d")
        return (date - base_date).days
    
    return column.apply(date_diff)

def convert_date_columns_to_int(dataframes, cols=['registration_init_time', 'expiration_date'], saving=True):
    count = 0
    total = len(dataframes)
    for df_name, df in dataframes.items():
        count += 1
        print("Starting {} ...".format(df_name))
        for col in cols:
            df[col+'_int'] = date_to_int(df[col])
        df = df.drop(columns=cols)
        dataframes[df_name] = df
        
        if saving:
            print("Saving {} ...".format(df_name))
            df.to_csv('music_data/'+ df_name + '_date_converted.csv')
        
        print("====== Done {} / {} ======".format(count, total))
    return dataframes


In [ ]:
# Converting all data
converted_dfs = convert_date_columns_to_int({
    'train_X': train_X,
    'valid_X': val_X,
    'test_X': test_X,
    'train_tiny_X': train_tiny_X,
    'val_tiny_X': val_tiny_X,
})
train_X = converted_dfs['train_X']
val_X = converted_dfs['valid_X']
test_X = converted_dfs['test_X']
train_tiny_X = converted_dfs['train_tiny_X']
val_tiny_X = converted_dfs['val_tiny_X']

Starting val_tiny_X ...
Saving val_tiny_X ...
====== Done 1 / 5 ======
Starting train_X ...
Saving train_X ...


In [30]:
train_X.head()

,Unnamed: 0,Unnamed: 0.1,msno,song_id,source_screen_name,source_system_tab,source_type,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,time,registration_init_time_int,expiration_date_int
0,0,2942719,8145,253733,16,6,8,267517.0,371,4252,8389,2681,9,0,0,2,1,0.296221,6193,6196
1,1,4875524,5224,145235,16,6,8,200620.0,371,34892,74276,26024,6,3,41,1,2,0.490781,5000,6475
2,2,6589819,5474,22231,11,0,7,213342.0,371,20609,27775,9110,9,0,0,2,1,0.663346,6150,6436
3,3,1172060,23177,70181,8,3,3,262246.0,371,44425,83027,34734,2,0,0,2,0,0.117982,5181,6498
4,4,2069395,3269,128141,12,2,2,310753.0,371,42400,81151,32836,2,0,0,2,2,0.208310,4393,6275


## Gradient Boosting

### LightGBM

In [19]:
def run_lgbc(train_X, train_y, val_X, val_y):
    params = {
        "objective" : "binary",
        "metric" : "binary_logloss", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)

    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return model, pred_val_y

#### Testing on tiny dataset

In [31]:
modelC, pred_val_y = run_lgbc(train_tiny_X, train_tiny_y['target'], val_tiny_X, val_tiny_y['target'])
print(classification_report(val_tiny_y['target'], modelC.predict(val_tiny_X)>0.5))
print(roc_auc_score(val_tiny_y['target'], modelC.predict(val_tiny_X)))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.651618
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.639451
              precision    recall  f1-score   support

         0.0       0.58      0.69      0.63       477
         1.0       0.66      0.55      0.60       523

   micro avg       0.62      0.62      0.62      1000
   macro avg       0.62      0.62      0.62      1000
weighted avg       0.62      0.62      0.61      1000

0.6819830761892165


#### Running large data

In [32]:
modelC, pred_val_y = run_lgbc(train_X, train_y['target'], val_X, val_y['target'])
print(classification_report(val_y['target'], modelC.predict(val_X)>0.5))
print(roc_auc_score(val_y['target'], modelC.predict(val_X)))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.6224
[200]	valid_0's binary_logloss: 0.616636
[300]	valid_0's binary_logloss: 0.612586
[400]	valid_0's binary_logloss: 0.609499
[500]	valid_0's binary_logloss: 0.606984
[600]	valid_0's binary_logloss: 0.604466
[700]	valid_0's binary_logloss: 0.602129
[800]	valid_0's binary_logloss: 0.60016
[900]	valid_0's binary_logloss: 0.598241
[1000]	valid_0's binary_logloss: 0.596447
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.596447
              precision    recall  f1-score   support

         0.0       0.67      0.70      0.68    732500
         1.0       0.69      0.66      0.67    742961

   micro avg       0.68      0.68      0.68   1475461
   macro avg       0.68      0.68      0.68   1475461
weighted avg       0.68      0.68      0.68   1475461

0.7449332780580912
